In [2]:
%load_ext autoreload
%autoreload 2

import datajoint as dj
dj.config["enable_python_native_blobs"] = True
dj.config['nnfabrik.schema_name'] = "nnfabrik_v1_tuning"

from nnfabrik.utility.dj_helpers import CustomSchema
from nnfabrik.main import *
schema = CustomSchema("nnfabrik_v1_tuning")

from insilico_stimuli.tables.main import ExperimentMethod, ExperimentSeed

Connecting pawelp@134.2.168.16:3306


In [3]:
method_fn = 'insilico_stimuli.optimizers.gradient_descent'
method_config = {'epochs': 10000, 'lr': 0.01, 'fixed_norm': 10}

ExperimentMethod()

method_fn name of the set function,method_hash hash of the set config,method_config set configuration object,method_ts UTZ timestamp at time of insertion,method_comment a short comment describing the set
insilico_stimuli.optimizers.bruteforce,34a3bdf5633d49c4e69c3ea63319b7ee,=BLOB=,2020-12-17 15:50:59,bf_batch_1000
insilico_stimuli.optimizers.bruteforce,531a53f0acae47bfa0c67acb2db6bd67,=BLOB=,2020-12-16 20:36:08,bf_batch_100
insilico_stimuli.optimizers.gradient_descent,4dfd6849154d32911f0f1229dfd9e505,=BLOB=,2021-01-15 16:22:15,grad_ep_10000_lr_0.01_std_0.1
insilico_stimuli.optimizers.gradient_descent,83547a0aba34d1a03c1f77378e2c5872,=BLOB=,2020-12-29 12:18:49,grad_ep_1000_lr_0.01_std_0.1
insilico_stimuli.optimizers.gradient_descent,e63e52df27ccc3c48d0b6153226d72da,=BLOB=,2021-01-21 11:55:09,"ep: 10k, lr: 0.01, norm: 10"
insilico_stimuli.optimizers.mse,4eddfbfba0262d08875860ffa23112e6,=BLOB=,2021-01-27 09:33:20,"ep: 10k, lr: 0.01, norm: 10"
nnidentify.experiment.experiment,d41d8cd98f00b204e9800998ecf8427e,=BLOB=,2020-12-17 15:31:41,experiment


Run the cell below as a job on the compute server

In [4]:
%%writefile run_scripts/gabor_run.py

import datajoint as dj
dj.config["enable_python_native_blobs"] = True
dj.config['nnfabrik.schema_name'] = "nnfabrik_v1_tuning"

from nnfabrik.utility.dj_helpers import CustomSchema
from nnfabrik.main import *
schema = CustomSchema("nnfabrik_v1_tuning")

from insilico_stimuli.tables.templates import ExperimentPerUnitTemplate

from nndichromacy.tables.from_mei import TrainedEnsembleModel
from nndichromacy.tables.from_mei import MEISelector
from nndichromacy.tables.measures import SignalToNoiseRatio

import numpy as np

@schema
class OptimalMouseGaborPerUnitExperiment(ExperimentPerUnitTemplate, dj.Computed):
    """MEI table template.

    To create a functional "MEI" table, create a new class that inherits from this template and decorate it with your
    preferred Datajoint schema. Next assign your trained model (or trained ensemble model) and your selector table to
    the class variables called "trained_model_table" and "selector_table". By default, the created table will point to
    the "MEIMethod" table in the Datajoint schema called "nnfabrik.main". This behavior can be changed by overwriting
    the class attribute called "method_table".
    """

    trained_model_table = TrainedEnsembleModel
    unit_table = MEISelector

dataset_key = {
    'dataset_hash': '72930dd1be6c229df4be82f74803262c' # Set the dataset here
}
    
np.random.seed(1000)
unit_keys = (SignalToNoiseRatio().Units() & dataset_key & 'unit_snr > 0.5').fetch('KEY')
unit_keys = [unit_keys[idx] for idx in np.random.choice(range(len(unit_keys)), size=150, replace=False)]

for key in unit_keys:
    key['ensemble_hash'] = '8a2aaa598935fa40a3f7db6f84209d5f' # Set ensemble hash here
    key['stimulus_hash'] = 'e478d1851e5a6170f854aae68719cd01'
    key['method_hash'] = 'e63e52df27ccc3c48d0b6153226d72da'
    key['dataset_hash'] = '72930dd1be6c229df4be82f74803262c' # Set dataset hash
    
OptimalMouseGaborPerUnitExperiment().populate(unit_keys, reserve_jobs=True, display_progress=True)

### Test the progress by running the cell below

In [8]:
selection = {
    'ensemble_hash': '97e2057bbafe6d713575d0527518276b', # Set ensemble hash here
    'stimulus_hash': 'e478d1851e5a6170f854aae68719cd01',
    'method_hash': 'e63e52df27ccc3c48d0b6153226d72da',
    'dataset_hash': '460f80e59cfde6ad5dca4b2c59caba3e', # Set dataset hash
}

OptimalMouseGaborPerUnitExperiment & selection

method_fn name of the set function,method_hash hash of the set config,stimulus_fn name of the set function,stimulus_hash hash of the set config,dataset_fn name of the dataset loader function,dataset_hash hash of the configuration object,ensemble_hash the hash of the ensemble,unit_id unique neuron identifier,data_key unique session identifier,seed experiment seed,output,score
insilico_stimuli.optimizers.gradient_descent,e63e52df27ccc3c48d0b6153226d72da,insilico_stimuli.stimuli.GaborSet,e478d1851e5a6170f854aae68719cd01,nndichromacy.datasets.static_loaders,460f80e59cfde6ad5dca4b2c59caba3e,97e2057bbafe6d713575d0527518276b,32,25133-4-16,1000,=BLOB=,3.17129
insilico_stimuli.optimizers.gradient_descent,e63e52df27ccc3c48d0b6153226d72da,insilico_stimuli.stimuli.GaborSet,e478d1851e5a6170f854aae68719cd01,nndichromacy.datasets.static_loaders,460f80e59cfde6ad5dca4b2c59caba3e,97e2057bbafe6d713575d0527518276b,32,25133-4-16,2000,=BLOB=,3.17137
insilico_stimuli.optimizers.gradient_descent,e63e52df27ccc3c48d0b6153226d72da,insilico_stimuli.stimuli.GaborSet,e478d1851e5a6170f854aae68719cd01,nndichromacy.datasets.static_loaders,460f80e59cfde6ad5dca4b2c59caba3e,97e2057bbafe6d713575d0527518276b,32,25133-4-16,3000,=BLOB=,3.17125
insilico_stimuli.optimizers.gradient_descent,e63e52df27ccc3c48d0b6153226d72da,insilico_stimuli.stimuli.GaborSet,e478d1851e5a6170f854aae68719cd01,nndichromacy.datasets.static_loaders,460f80e59cfde6ad5dca4b2c59caba3e,97e2057bbafe6d713575d0527518276b,32,25133-4-16,4000,=BLOB=,3.17108
insilico_stimuli.optimizers.gradient_descent,e63e52df27ccc3c48d0b6153226d72da,insilico_stimuli.stimuli.GaborSet,e478d1851e5a6170f854aae68719cd01,nndichromacy.datasets.static_loaders,460f80e59cfde6ad5dca4b2c59caba3e,97e2057bbafe6d713575d0527518276b,32,25133-4-16,5000,=BLOB=,3.17139
insilico_stimuli.optimizers.gradient_descent,e63e52df27ccc3c48d0b6153226d72da,insilico_stimuli.stimuli.GaborSet,e478d1851e5a6170f854aae68719cd01,nndichromacy.datasets.static_loaders,460f80e59cfde6ad5dca4b2c59caba3e,97e2057bbafe6d713575d0527518276b,34,25133-4-16,1000,=BLOB=,5.61002
insilico_stimuli.optimizers.gradient_descent,e63e52df27ccc3c48d0b6153226d72da,insilico_stimuli.stimuli.GaborSet,e478d1851e5a6170f854aae68719cd01,nndichromacy.datasets.static_loaders,460f80e59cfde6ad5dca4b2c59caba3e,97e2057bbafe6d713575d0527518276b,34,25133-4-16,2000,=BLOB=,5.61075
insilico_stimuli.optimizers.gradient_descent,e63e52df27ccc3c48d0b6153226d72da,insilico_stimuli.stimuli.GaborSet,e478d1851e5a6170f854aae68719cd01,nndichromacy.datasets.static_loaders,460f80e59cfde6ad5dca4b2c59caba3e,97e2057bbafe6d713575d0527518276b,34,25133-4-16,3000,=BLOB=,5.61026
insilico_stimuli.optimizers.gradient_descent,e63e52df27ccc3c48d0b6153226d72da,insilico_stimuli.stimuli.GaborSet,e478d1851e5a6170f854aae68719cd01,nndichromacy.datasets.static_loaders,460f80e59cfde6ad5dca4b2c59caba3e,97e2057bbafe6d713575d0527518276b,34,25133-4-16,4000,=BLOB=,5.61127
insilico_stimuli.optimizers.gradient_descent,e63e52df27ccc3c48d0b6153226d72da,insilico_stimuli.stimuli.GaborSet,e478d1851e5a6170f854aae68719cd01,nndichromacy.datasets.static_loaders,460f80e59cfde6ad5dca4b2c59caba3e,97e2057bbafe6d713575d0527518276b,34,25133-4-16,5000,=BLOB=,5.61009


### Export the gabors once they have finnished optimising

In [19]:
import pickle

unit_keys, outputs, scores = (OptimalMouseGaborPerUnitExperiment() & selection).fetch('KEY', 'output', 'score', order_by='unit_id')

for unit_key, output in zip(unit_keys, outputs):
    unit_key['gabor'] = output['image'].squeeze()

curated_unit_keys = []
max_seed = scores.reshape(150, 5).argmax(1)

for gabor_idx, idx in enumerate(range(0, len(unit_keys), 5)):
    curated_unit_keys.append(unit_keys[idx + max_seed[gabor_idx]])
    
with open('./results/gabors.pkl', 'wb') as f:
    pickle.dump(curated_unit_keys, f)